In [2]:
import numpy as np
import os
import scipy.io
import nibabel as nib
import re
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import random
import pandas as pd
from math import isnan
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import SpectralClustering
import itertools
from itertools import combinations 
from scipy.spatial import distance
from sklearn.neighbors import radius_neighbors_graph as rng
import pickle
import _pickle as cPickle
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, jaccard_similarity_score, adjusted_mutual_info_score, adjusted_rand_score, fowlkes_mallows_score

# Unpacking data 

In [1]:
def all_data(all_directories, sides):
    #Function which takes list of directories to data and sides and returns dictionary with lists of files with rqas
    mask_directories = os.listdir(all_directories)[:] #directories to all possible masks
    dict_rqa_files={mask_name:[os.listdir(all_directories+mask_name+'/'+ hemisphere+'/') for hemisphere in sides] for mask_name in mask_directories if mask_name[0]!='.'} # dictionary with keys: names of masks and values: list of directories containing mask name and side
    dict_rqa_files = {key:[[all_directories+key+'/'+sides[num] +'/'+one_file for one_file in value_side if one_file[0]!='.'] for num, value_side in enumerate(values)] for key, values in dict_rqa_files.items()}
    #dictionary with mask names as keys and list of directories to .mat files depending on the side and subjects
    return(dict_rqa_files)

In [ ]:
decode_measure = {1:'DET', 2:'AverageDiagonal', 3:'MaxLine', 4:'entropy', 5:'laminarity', 6:'TT'}

In [ ]:
def create_matrices(all_directories, sides = ['Left', 'Right'], name_dict = 'rqaValuesMatrix', measures = [1,2,3,4,5,6], decode_measure = decode_measure):
    #function taking the directory with files contatining data in.mat format with RQA measures valeus (all_directories),
    #list with name left and right for the side where amygdala is located,
    #name of matrix where rq`a values are located
    #numbers of measures to take 
    #and dictionary for decoding numbers into RQA measures names
    #and returning 
    dict_files = all_data(all_directories, sides) #dictionary with mask names as keys and list of directories to .mat files depending on the side and subjects
    matrices_to_cluster = {(keys,decode_measure[measure]):[create_matrix(value_side, measure, name_dict) for value_side in values] for keys, values in dict_files.items() for measure in measures}
    #dictionary with keys as names of mask and name of given measure and [list of matrices as values]
    return(matrices_to_cluster)

In [ ]:
def create_matrix(list_data, measure, name_in_dict):
    #function taking the directory with files contatining data in.mat format with RQA measures valeus (list_data)
    #measure number taken into account
    #and name for the particular matrix with data
    # and returnarray with data for a single subject
    data_matrix2 = np.array([scipy.io.loadmat(data_matrix)[name_in_dict][:,measure] for data_matrix in list_data])
    return(data_matrix2)

# Normalization

In [ ]:
def normalize_matrices(dict_matrices):
    #function takes dictionary from create_matrices function as input and return dictionary with standardized and non-standardize values
    nonstandard_matrices={(key, 'nonstandard'):value for key, value in dict_matrices.items()}
    standard_matrices = {(key, 'standard'):[((side_value - np.mean(side_value))/np.std(side_value)) for side_value in value] for key,value in dict_matrices.items()}
    standard_matrices.update(nonstandard_matrices)
    return(standard_matrices)

# Clustering

In [ ]:
def coordinates_function_dict(coordinatesDir, list_masks = ['Bach'], side_list = ['Left', 'Right']):
    #function takes directory to files with mask of amygdala in MNI
    #and return dict with matrices and coordinates
    link_dict = {mask_name:[coordinatesDir + side + '/' + mask_name + 'Mask.nii.gz' for side in side_list] for mask_name in list_masks}
    matrices_dict = {keys:[nib.load(side).get_data() for side in values] for keys, values in link_dict.items()}
    coordinates_dict = {keys:[[(x_coor, y_coor, z_coor) for x_coor, x in enumerate(side) for y_coor, y in enumerate(x) for z_coor, z in enumerate(y) if z!=0] for side in values] for keys, values in matrices_dict.items()}
    return([matrices_dict, coordinates_dict])

In [ ]:
def dict_clustering(result_dict, type_clustering = ['Hierarchy'], num_clustering = [2], linkage = 'ward', neighbors = True, coordinates_dict = coordinates_dict, num_repetitions = 1000):
    #function takes dictionary with results and return dictionary after performing clustering
    clustering_numerous = {keys:{(type_clust, num_clust):[[clustering_typeF(np.transpose(side_values), type_clust, num_clust, linkage, neighbors, coordinates_dict[keys[0][0]][num]) for i in range(num_repetitions)] for num, side_values in enumerate(values)] for type_clust in type_clustering for num_clust in num_clustering} for keys, values in result_dict.items()}
    clustering_final_labels = {keys:{keys2:[np.round(np.mean(side, axis = 0)) for num, side in enumerate(arrays)] for keys2, arrays in values.items()} for keys, values in clustering_numerous.items()}
    return(clustering_final_labels)

In [ ]:
def clustering_typeF(dataset, typeClustering, numClust, linkage, neighbors, coordinates):
    #function takes dataset to cluster and return return appropraite labels after clustering
    if (typeClustering == 'KMeans'):
        labels = KMeans_clustering(dataset, numClust)
    elif (typeClustering == 'Hierarchy'):
        labels = Ward_hierarchical_clustering(dataset, numClust, linkage, neighbors, coordinates)
    elif (typeClustering == 'Spectral'):
        labels = Spectral_clustering(dataset, numClust)
    lengthLabels = [np.sum([1 for j in labels if i==j]) for i in range(1,numClust+1)]
    lengthLabelsOrdered = sorted(lengthLabels, reverse=True)
    if lengthLabelsOrdered[0]!=lengthLabelsOrdered[1]:
        labels = changeLabels(labels, lengthLabels, lengthLabelsOrdered)
    return(labels)

In [ ]:
def changeLabels(labels, lengthLabels, lengthLabelsOrdered):
    #function ordered clusters - 1 - the biggest cluster, 2 - 2nd biggest cluster etc
    original_labeling = {(num+1):i for num, i in enumerate(lengthLabels)}
    original_labeling_reverse = {value:key for key,value in original_labeling.items()}
    if (len(original_labeling.items())!=len(original_labeling_reverse.items())):
        return(np.zeros(len(labels)))
    appropriate_labeling = {(num+1):i for num, i in enumerate(lengthLabelsOrdered)} 
    
    
    for j in range(len(lengthLabels)):
        if original_labeling[j+1] != appropriate_labeling[j+1]:
             for key, value in appropriate_labeling.items():
                if value == original_labeling[j+1]:
                        labels[labels==original_labeling_reverse[value]] = key + len(lengthLabels)
                        
    for k in np.unique(labels):
        if k > len(lengthLabels):
            labels[labels==k]=k-len(lengthLabels)
    return(labels)

#performing KMeans Clustering

def KMeans_clustering(i, n_clusters):    
    RQA_matrix = np.array(i)
    K_Means = KMeans(n_clusters).fit_predict(RQA_matrix) + 1
    return(K_Means)

#performing Agglomerative Clustering (Ward hierarchical clustering)

def Ward_hierarchical_clustering(i, n_clusters, linkage, neighbors, coordinates):
    RQA_matrix = np.array(i)       
    if neighbors == True:
        Ward = AgglomerativeClustering(n_clusters, linkage = linkage, ).fit_predict(RQA_matrix) + 1
    else: 
        connectivity = rng(coordinates, radius =1, mode="connectivity", metric = "minkowski", p=2, include_self=None)
        Ward = AgglomerativeClustering(n_clusters, linkage = linkage, connectivity = connectivity).fit_predict(RQA_matrix) + 1
    return(Ward)

#performing Spectral Clustering

def Spectral_clustering(i, n_clusters):
    j = scipy.spatial.distance.pdist(i)
    k = scipy.spatial.distance.squareform(j)
    mean_dist = np.mean(k)
    final_matrix = np.exp(-k/(2*(mean_dist**2)))
    RQA_matrix = np.array(final_matrix)
    Spectral = SpectralClustering(n_clusters, affinity = 'precomputed').fit(RQA_matrix) 
    Spectral = Spectral.labels_ + 1
    return(Spectral)

# Validation

In [ ]:
def all_validations(clustering_labels_dict, coordinates_dict, matrices_dict, groundtruthDicts):
    code_side = {0:'Left', 1:'Right'} #zakodowanie strony
    #stability_indicator = {keys:{keys2:[np.mean([max({j:np.sum([1 for i in np.array(side)[:,voxel] if i==j])/num_repetitions for j in np.unique(np.array(side)[0,:])}.values()) for voxel in range(np.array(side).shape[1])]) for num, side in enumerate(arrays)] for keys2,arrays in values.items()} for keys, values in clustering_numerous.items()}
    internal_validation_dict = {keys:{keys2:[internal_validation_test(side_vector, coordinates_dict[keys[0][0]][num]) for num, side_vector in enumerate(arrays)] for keys2, arrays in values.items()} for keys, values in clustering_labels_dict.items()} 
    external_validation_dict = {keys:{keys2:{keys3:[external_validation_measures(side_vector, keys[0][0], code_side[num], groundtruthDicts[keys3][code_side[num]],coordinates_dict, matrices_dict) for num, side_vector in enumerate(arrays)] for keys3, gtValues in groundTruthDicts.items()} for keys2, arrays in values.items()} for keys, values in clustering_labels_dict.items()}
    external_validation_dict = {keys:{keys2:{keys3:{keys4:[dict_measures_side[keys4] for dict_measures_side in values3] for dict_measures_side in values3 for keys4, values4 in dict_measures_side.items()} for keys3, values3 in arrays.items()} for keys2, arrays in values.items()} for keys, values in external_validation_dict.items()}
    external_validation_dict = {keys:{keys2:{keys3:{keys4:{keys5:[side_measures2[keys5] for side_measures2 in values4] for side_measures in values4 for keys5, values5 in side_measures.items()} for keys4, values4 in values3.items()} for keys3, values3 in arrays.items()} for keys2, arrays in values.items()} for keys, values in external_validation_dict.items()}
    all_validation_results = {keys:{keys2:[values2, internal_validation_dict[keys][keys2], external_validation_dict[keys][keys2]] for keys2, values2 in values.items()} for keys, values in clustering_labels_dict.items()}
    return(all_validation_results)

In [ ]:
groundTruthDicts = {'Julich':{'Left':'/Users/kbielski/Documents/Amygdala/Results Previous/AmygdalaReComputedData/ExternalValidationMasks/Julich/LeftClassificationJulich.nii.gz',
                              'Right':'/Users/kbielski/Documents/Amygdala/Results Previous/AmygdalaReComputedData/ExternalValidationMasks/Julich/RightClassificationJulich.nii.gz'},
                   'Bach':{'Left':'/Users/kbielski/Documents/Amygdala/Results Previous/AmygdalaReComputedData/ExternalValidationMasks/Bach/BachmaskResampledLeft.nii.gz',
                           'Right':'/Users/kbielski/Documents/Amygdala/Results Previous/AmygdalaReComputedData/ExternalValidationMasks/Bach/BachmaskResampledRight.nii.gz'}}

In [ ]:
def internal_validation_test(clustering_vector, clustering_coordinates):
    labels = np.unique(clustering_vector)
    metrica = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(np.array(clustering_coordinates), 'euclidean'))
    cluster_coordinates = np.array([np.array([list(clustering_coordinates[num]) for num,clus in enumerate(clustering_vector) if i==clus]) for i in labels])
    pairs_of_clusters = [[num for num, label in enumerate(clustering_vector) if label == clust] for clust in labels]
    distance_matrices = [scipy.spatial.distance.squareform(scipy.spatial.distance.pdist([clustering_coordinates[indices] for indices in clust])) for clust in pairs_of_clusters]
    intra_cluster_distance = [np.sum(distance)/2 for distance in distance_matrices]
    inter_cluster_distance = [np.sum([scipy.spatial.distance.euclidean(clustering_coordinates[indices_first], clustering_coordinates[indices_second]) for indices_first in indicesX for indices_second in indicesY]) 
                               for num, indicesX in enumerate(pairs_of_clusters) for indicesY in pairs_of_clusters[num+1:] if num!=(len(pairs_of_clusters)-1)]
    all_inter_cluster_indices = [[(num, num1+num+1) for num1, indicesY in enumerate(pairs_of_clusters[num+1:]) if num!=(len(pairs_of_clusters)-1)] for num, indicesX in enumerate(pairs_of_clusters)]
    labelki=[[(first_label, second_label) for pairs in all_inter_cluster_indices for (first_label, second_label) in pairs if first_label == (unique_label-1) or second_label == (unique_label-1)] for unique_label in labels]
    ordered_labelki = [(num, num1+num+1) for num, indicesX in enumerate(pairs_of_clusters) for num1, indicesY in enumerate(pairs_of_clusters[num+1:])]
    inter_cluster_distances_all = [sum([inter_cluster_distance[num] for (indX, indY) in indices for num,(indX_2, indY_2) in enumerate(ordered_labelki) if indX==indX_2 and indY == indY_2]) for indices in labelki]
    intra_edges = [int(scipy.special.binom(len(coordinates),2)) for coordinates in cluster_coordinates]
    inter_edges = [len(coordinates)*len(coordinatesNext) for num,coordinates in enumerate(cluster_coordinates) for coordinatesNext in cluster_coordinates[num+1:]]
    Beta_CV = (np.sum(intra_cluster_distance)/np.sum(intra_edges))/(np.sum(inter_cluster_distance)/np.sum(inter_edges))
    Normalized_Cut = np.sum([1/((intra_distance/inter_cluster_distances_all[num])+1) for num, intra_distance in enumerate(intra_cluster_distance)])/len(labels)
    if (len(labels)==1):
        silhouette_coefficient = 0
    else:
        silhouette_coefficient = silhouette_score(metrica, clustering_vector, metric='precomputed')
    return([Beta_CV, Normalized_Cut, silhouette_coefficient])

In [ ]:
def external_validation_measures(classifications, mask_name_main, side, ground_truths, coordinates_dict, matrices_dict):
    [maskLabelsLimited, gtLabelsLimited, maskLabelsFull, gtLabelsFull] = mask_to_evaluate(classifications, mask_name_main, side, ground_truths, coordinates_dict, matrices_dict)
    #purityLimited = {keys:[{one_label:np.sum([1 for x in gtLabelsLimited[keys][[num for num, label in enumerate(labels_limited) if label==num_clust] for num_clust in np.unique(labels_limited)] if x==one_label]) for one_label in np.unique(gtLabelsLimited[keys][[num for num, label in enumerate(labels_limited) if label==num_clust] for num_clust in np.unique(labels_limited)])} 
    lengthLimited = len(maskLabelsLimited) 
    lengthFull = len(maskLabelsFull)
    names_list = ['Purity','FScore','MutualInfo','Jaccard','Fowlkes-Mallows']
    purity = {'Limited': compute_purity(maskLabelsLimited, gtLabelsLimited, lengthLimited), 'Full': compute_purity(maskLabelsFull, gtLabelsFull, lengthFull)}
    Fscore = {'Limited': compute_f_score(maskLabelsLimited, gtLabelsLimited), 'Full': compute_f_score(maskLabelsFull, gtLabelsFull)}
    mutualInfo = {'Limited': compute_mutual_info(maskLabelsLimited, gtLabelsLimited), 'Full': compute_mutual_info(maskLabelsFull, gtLabelsFull)}
    jaccard = {'Limited': compute_jaccard(maskLabelsLimited, gtLabelsLimited), 'Full': compute_jaccard(maskLabelsFull, gtLabelsFull)}
    fowkles = {'Limited': compute_fawlkes(maskLabelsLimited, gtLabelsLimited), 'Full': compute_fawlkes(maskLabelsFull, gtLabelsFull)}
    wholeExternalMeasures = {names_list[0]:purity,
                             names_list[1]:Fscore,
                             names_list[2]:mutualInfo,
                             names_list[3]:jaccard,
                             names_list[4]:fowkles} 
    return(wholeExternalMeasures)

In [ ]:
def compute_fawlkes(maskLabels, gtLabels):
    fawlkes = fowlkes_mallows_score(gtLabels, maskLabels)
    return(fawlkes)

In [ ]:
def compute_jaccard(maskLabels, gtLabels):
    jaccard = jaccard_similarity_score(gtLabels, maskLabels)
    return(jaccard)

In [ ]:
def compute_f_score(maskLabels, gtLabels):
    #print(len(maskLabels))
    #print(len(gtLabels))
    limited_Fscore=f1_score(gtLabels, maskLabels, average='micro')
    return(limited_Fscore)

In [ ]:
def compute_mutual_info(maskLabels, gtLabels):
    mutual_info = adjusted_mutual_info_score(gtLabels, maskLabels) 
    return(mutual_info)

In [ ]:
def compute_purity(maskLabels, gtLabels, length):
    indices_purity =[gtLabels[[num for num, label in enumerate(maskLabels) if label==num_clust]] for num_clust in np.unique(maskLabels)] 
    purity_dict_dict=[{unique_label:np.sum([1 for label in one_cluster if label==unique_label]) for unique_label in np.unique(one_cluster)} for num, one_cluster in enumerate(indices_purity)] 
    purity_dict = (1/length)*np.sum([sorted(list(dict_cluster.items()),key=lambda x: x[1], reverse = True)[0][1] for dict_cluster in purity_dict_dict]) 
    return(purity_dict)

In [ ]:
def mask_to_evaluate(classifications, mask_name_main, side, ground_truths, coordinates_dict, matrices_dict):
    side_decoder = {'Left':0, 'Right':1}
    groundTruthFiles = open_ground_truth_file(ground_truths)
    dict_coordinates = np.array(coordinates_dict[mask_name_main][side_decoder[side]])
    [maskLabelsLimited, gtLabelsLimited] = limited_masks(groundTruthFiles, dict_coordinates, classifications, matrices_dict, mask_name_main, side_decoder[side])
    [maskLabelsFull, gtLabelsFull] = full_masks(groundTruthFiles, dict_coordinates, classifications, matrices_dict, mask_name_main, side_decoder[side])
    return([maskLabelsLimited, gtLabelsLimited, maskLabelsFull, gtLabelsFull])

In [ ]:
def limited_masks(groundTruthFiles, dict_coordinates, classifications, coordinatesDir, mask_name, side):
    limited_mask_GT = [num for num, (x_1, y_1, z_1) in enumerate(groundTruthFiles[1]) for (x_2, y_2, z_2) in dict_coordinates if x_1 == x_2 and y_1 == y_2 and z_1 == z_2] 
    limited_mask_Original = [num for num, (x_1, y_1, z_1) in enumerate(dict_coordinates) for (x_2, y_2, z_2) in groundTruthFiles[1] if x_1 == x_2 and y_1 == y_2 and z_1 == z_2] 
    classification_limited_GT = [groundTruthFiles[0][limited_mask_GT],groundTruthFiles[1][limited_mask_GT]]
    classification_limited_Original = [np.array(classifications)[limited_mask_Original], dict_coordinates[limited_mask_Original]]
    maskLimited = labeling_limited(classification_limited_Original, matrices_dict, mask_name, side)
    gtLimited = labeling_limited(classification_limited_GT, matrices_dict, mask_name, side)
    return([maskLimited, gtLimited])

In [ ]:
def full_masks(groundTruthFiles, dict_coordinates, classifications, matrices_dict, mask_name, side):
    notInOriginal = [[x_1, y_1, z_1] for [x_1, y_1, z_1] in groundTruthFiles[1] if np.sum([1 for [x_2, y_2, z_2] in dict_coordinates if [x_1, y_1, z_1]!=[x_2, y_2, z_2]])==len(dict_coordinates)] 
    notInGT = [[x_1, y_1, z_1] for [x_1, y_1, z_1] in dict_coordinates if np.sum([1 for [x_2, y_2, z_2] in groundTruthFiles[1] if [x_1, y_1, z_1]!=[x_2, y_2, z_2]]) == len(groundTruthFiles[1])] 
    notInOriginalInd = [num for num, [x_1, y_1, z_1] in enumerate(groundTruthFiles[1]) if np.sum([1 for [x_2, y_2, z_2] in dict_coordinates if [x_1, y_1, z_1]!=[x_2, y_2, z_2]])==len(dict_coordinates)] 
    notInGTInd = [num for num, [x_1, y_1, z_1] in enumerate(dict_coordinates) if np.sum([1 for [x_2, y_2, z_2] in groundTruthFiles[1] if [x_1, y_1, z_1]!=[x_2, y_2, z_2]]) == len(groundTruthFiles[1])]
    coordinatesFullOriginal = [list(j) for i in [dict_coordinates, notInOriginal] for j in i] 
    coordinatesFullGT = [list(j) for i in [groundTruthFiles[1], notInGT] for j in i] 
    classificationsOriginal = [j for i in [classifications,[max(classifications)+1]*len(notInOriginalInd)] for j in i] 
    classificationsGT = [j for i in [groundTruthFiles[0],[max(groundTruthFiles[0])+1]*len(notInGT)] for j in i] 
    labelsOriginal = labeling_full(classificationsOriginal, coordinatesFullOriginal, matrices_dict, mask_name, side)
    labelsGT = labeling_full(classificationsGT, coordinatesFullGT, matrices_dict, mask_name, side)
    return([labelsOriginal, labelsGT])

In [ ]:
def labeling_limited(classifications, matrices_dict, mask_name, side):
    #print('Labeling:')
    #print(classifications[0])
    #print(classifications[1])
    side_decoder = {'Left':0, 'Right':1}
    data_sety=matrices_dict[mask_name][side]
    empty_spaces = np.zeros(data_sety.shape)
    for num,[x_1, y_1, z_1] in enumerate(classifications[1]):
        empty_spaces[x_1][y_1][z_1]=classifications[0][num]
    new_labeling = empty_spaces[empty_spaces!=0]
    return(new_labeling)

In [ ]:
def labeling_full(classifications, coordinates, matrices_dict, mask_name, side):
    side_decoder = {'Left':0, 'Right':1}
    data_sety=matrices_dict[mask_name][side]
    empty_spaces = np.zeros(data_sety.shape) 
    for num,[x_1, y_1, z_1] in enumerate(coordinates):
        empty_spaces[x_1][y_1][z_1]=classifications[num]
    new_labeling =empty_spaces[empty_spaces!=0]
    return(new_labeling)

In [ ]:
def open_coordinates(side, coordinatesDir, mask_name_main):
    open_mask=nib.load(coordinatesDir + side +'/'+mask_name_main+'Mask.nii.gz')
    load_data = open_mask.get_data()
    coordinates = np.array([(x,y,z) for x, x_values in enumerate(load_data)
                                    for y, y_values in enumerate(x_values)
                                    for z, z_values in enumerate(y_values) if z_values!=0])
    return(coordinates)

In [ ]:
def open_ground_truth_file(file_ground_truth):
    loaded_file = nib.load(file_ground_truth)
    data_file = loaded_file.get_data()
    classificationGroundTruth = np.array([int(round(i)) for i in data_file[data_file!=0]])
    coordinatesGroundTruth = np.array([(x,y,z) for x, value_x in enumerate(data_file) 
                                               for y, value_y in enumerate(value_x)
                                               for z, value_z in enumerate(value_y) if value_z!=0])
    return([classificationGroundTruth, coordinatesGroundTruth])

In [ ]:
def distances_test_original(testDataset, clusterTest):
    intra_cluster_matrices = metrices_for_computing_distances(testDataset, clusterTest)
    intra_cluster_distances = np.array([[[distance.euclidean(subset[:,ind_vect_one], subset[:,ind_vect_two]) for ind_vect_two in range(ind_vect_one+1, subset.shape[1]) if ind_vect_one!=subset.shape[1]-1] for ind_vect_one in range(subset.shape[1])] for subset in intra_cluster_matrices])
    intra_cluster_sum_distances = [np.sum(two_array) for two_array in [np.sum(one_array) for one_array in intra_cluster_distances]]
    intra_cluster_number_connections = [sum([len(two_array) for two_array in one_array]) for one_array in intra_cluster_distances]
    intra_relative_sum_distances = [distances_intra/intra_cluster_number_connections[num] for num, distances_intra in enumerate(intra_cluster_sum_distances)]
    return(intra_relative_sum_distances)

In [ ]:
def metrices_for_computing_distances(testDataset, clusterTest):
    inter_cluster_indices = [[index for index, num_label in enumerate(clusterTest) if i == num_label] for i in np.unique(clusterTest)]
    inter_cluster_matrices = [testDataset[:,indices] for indices in inter_cluster_indices]
    return(inter_cluster_matrices)

In [ ]:
def distances_test_inter(testDataset, clusterTest):
    inter_cluster_matrices = metrices_for_computing_distances(testDataset, clusterTest)
    inter_cluster_distances = np.array([[[[distance.euclidean(matrix_one[:,vect_one], matrix_two[:,vect_two])] for vect_two in range(matrix_two.shape[1]) for vect_one in range(matrix_one.shape[1])] for matrix_two in inter_cluster_matrices[num+1:]] for num, matrix_one in enumerate(inter_cluster_matrices) if (num!=len(inter_cluster_matrices)-1)])
    inter_cluster_sum_distances = [[np.sum(two_comparison) for two_comparison in one_comparison] for one_comparison in inter_cluster_distances]
    inter_cluster_numbers = [[len(two_comparison) for two_comparison in one_comparison] for one_comparison in inter_cluster_distances]
    inter_cluster_relative = [[second_distance/inter_cluster_numbers[num][num2] for num2, second_distance in enumerate(first_distance)] for num, first_distance in enumerate(inter_cluster_sum_distances)]
    inter_cluster_relative = list(itertools.chain(*inter_cluster_relative))
    return(inter_cluster_relative)
    #inter_cluster_distances = [[[distance.euclidean(vect_one, vect_two) for vect_two in matrix_two for vect_one in matrix_one] for matrix_two in inter_cluster_matrices[num+1]] for num, matrix_one in enumerate(inter_cluster_matrices)]
    #return(inter_cluster_distances)
    #inter_cluster_distances = np.mean([distance.euclidean(i,j) for j in second_dataset for i in first_dataset for second_dataset in inter_cluster_matrices[num+1] for num, first_dataset in enumerate(inter_cluster_matrices) if (num!=len(inter_cluster_matrices)+1)])
    

In [ ]:
def change_dict_validation(dict_validation):
    modified_dict_validation={keys:[np.mean(values[0]), np.mean(values[1]), values[2], values[3]] for keys, values in dict_validation.items()}
    return(modified_dict_validation)

# Selecting solution

In [ ]:
#<list>.sort_values(by = [2], ascending = True)

# Comparing entropy between subdivisions

In [ ]:
def take_clustering(dataset, measures, algorithm):
    [left_clustering, right_clustering] = dataset[measures][algorithm]
    return([left_clustering, right_clustering])

In [ ]:
def take_index_label(clustering, label):
    indices = np.array([num for num, x in enumerate(clustering) if x==label])
    return(indices)

In [ ]:
def take_raw_data(raw_dataset, measures):
    [left_raw_data, right_raw_data] = raw_dataset[measures]
    return([left_raw_data, right_raw_data])

In [ ]:
def take_raw_data_cluster(raw_data, indices):
    values_one_cluster = raw_data[:,indices]
    return(values_one_cluster)

In [ ]:
def take_unique_labels(clustering):
    labels = np.unique(clustering)
    return(labels)

In [ ]:
def indices_of_parts(clustering, labels):
    indices_all = [take_index_label(clustering, one_label) for one_label in labels]
    return(indices_all)

In [ ]:
def take_data_cluster(raw_data_one, indices_list):
    values=[raw_data_one[:,indices] for indices in indices_list]
    return(values)

In [ ]:
def open_cluster_dataset(directory):
    cluster_dataset=np.load(directory, allow_pickle=True)
    return(cluster_dataset.item())

In [ ]:
def open_raw_datset(directory_raw):
    raw_dataset = np.load(directory_raw, allow_pickle=True)
    return(raw_dataset.item())

In [ ]:
def open_datasets(directory, directory_raw):
    [cluster_dataset, raw_dataset] = [open_cluster_dataset(directory), open_raw_datset(directory_raw)]
    return([cluster_dataset, raw_dataset])

In [ ]:
def take_data(directory, directory_raw, measures, algorithm):
    [cluster_dataset, raw_dataset] = open_datasets(directory, directory_raw)
    [cluster_data, raw_data]=[take_clustering(cluster_dataset, measures, algorithm), take_raw_data(raw_dataset, measures)]
    return([cluster_data, raw_data])

In [ ]:
def take_data_side(directory, directory_raw, measures, algorithm, side):
    [cluster_data, raw_data]=take_data(directory, directory_raw, measures, algorithm)
    return(cluster_data[side], raw_data[side])

In [ ]:
def take_one_side(directory, directory_raw, measures, algorithm, sides):
    all_datasets =[take_data_side(directory, directory_raw, measures, algorithm, one_side) for one_side in sides]
    return(all_datasets)

In [ ]:
def take_labels(directory, directory_raw, measures, algorithm, sides):
    unique_lables=[take_unique_labels(one_dataset[0]) for one_dataset in take_one_side(directory, directory_raw, measures, algorithm, sides)]
    return(unique_lables)

In [ ]:
def take_indices(directory, directory_raw, measures, algorithm, sides):
    indices=[indices_of_parts(one_dataset[0], take_labels(directory, directory_raw, measures, algorithm, sides)[num]) for num, one_dataset in enumerate(take_one_side(directory, directory_raw, measures, algorithm, sides))]
    return(indices)

In [ ]:
def take_values(directory, directory_raw, measures, algorithm, sides):
    values=[[one_dataset[1][:,num2labels] for num2labels in take_indices(directory, directory_raw, measures, algorithm, sides)[num]] for num, one_dataset in enumerate(take_one_side(directory, directory_raw, measures, algorithm, sides))]
    return(values)

In [ ]:
def compute_means(directory, directory_raw, measures, algorithm, sides):
    means_values = [[np.mean(cluster_values, axis = 0) for cluster_values in one_side_set] for one_side_set in take_values(directory, directory_raw, measures, algorithm, sides)]
    return(means_values)

In [ ]:
def levene_test_onetwo(directory, directory_raw, measures, algorithm, sides):
    results_test = [scipy.stats.levene(one_data_means[0], one_data_means[1]) for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    return(results_test)

In [ ]:
def t_test(directory, directory_raw, measures, algorithm, sides):
    results_test = [scipy.stats.ttest_ind(one_data_means[0], one_data_means[1], equal_var=False) for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    return(results_test)

In [ ]:
def normality_test_onetwo(directory, directory_raw, measures, algorithm, sides):
    results_test = [[scipy.stats.normaltest(normality) for normality in one_data_means] for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    return(results_test)

In [ ]:
def normality_test_leftright(directory, directory_raw, measures, algorithm, sides):
    leftright_test = [scipy.stats.normaltest(np.concatenate(one_data_means)) for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    return(leftright_test)

In [ ]:
def levene_test_leftright(directory, directory_raw, measures, algorithm, sides):
    leftright_test = [np.concatenate(one_data_means) for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    levene=scipy.stats.levene(leftright_test[0], leftright_test[1])
    return(levene)

In [ ]:
def ttest_test_leftright(directory, directory_raw, measures, algorithm, sides):
    leftright_test = [np.concatenate(one_data_means) for one_data_means in compute_means(directory, directory_raw, measures, algorithm, sides)]
    ttest=scipy.stats.ttest_ind(leftright_test[0], leftright_test[1], equal_var=True)
    return(ttest)

# Visualization 

In [ ]:
x_labels = ['Left VL', 'Right VL', 'Left DM', 'Right DM']

In [ ]:
mean2018 = [np.mean(left2018_one), np.mean(right2018_one), np.mean(left2018_two), np.mean(right2018_two)]
std2018 = [np.std(left2018_one)/np.sqrt(len(left2018_one)), np.std(right2018_one)/np.sqrt(len(right2018_one)), np.std(left2018_two)/np.sqrt(len(left2018_two)), np.std(right2018_two)/np.sqrt(len(right2018_two))]
mean2019 = [np.mean(left2019_one), np.mean(right2019_one), np.mean(left2019_two), np.mean(right2019_two)]
std2019 = [np.std(left2019_one)/np.sqrt(len(left2019_one)), np.std(right2019_one)/np.sqrt(len(right2019_one)), np.std(left2019_two)/np.sqrt(len(left2018_two)), np.std(right2019_two)/np.sqrt(len(right2019_two))]

In [ ]:
fig, ax = plt.subplots()
ax.axhline(y =0, color = 'black', linewidth = .5)
ax.bar(x_labels, mean2019, yerr=std2019, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Value of Shanon\'s entropy')
ax.set_xlabel('Amygdala Subdivision')
ax.plot([0,0, 2, 2], [0.33, 0.34, 0.34, 0.33], linewidth=1, color='k')
ax.text(1,0.34,"***")
#ax.plot([2, 2, 3, 3], [-0.66, -0.67, -0.67, -0.66], linewidth=1, color='k')
#ax.text(2.5,-0.715,"*")
ax.plot([1, 1, 3, 3], [-0.7, -0.71, -0.71, -0.7], linewidth=1, color='k')
ax.text(2,-0.77,"***")